In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'/home/nina/PycharmProjects/INSIDe/inside_demonstrator/analysis'

In [41]:
experiment_name = "last_day/MittelstarkerRegen"

In [42]:
data_directory = f"../tandler_output/preprocessed/{experiment_name}"
try:
    os.makedirs(data_directory)
except:
    print("Folder structure already exists")

Folder structure already exists


### INSIDe substance results

In [35]:
file_directory = f"../tandler_output/original/{experiment_name}/"
sampling_files = [el for el in os.listdir(file_directory) if "Sampling_COV_INSIDe" in el]
sampling_curves_files = [el for el in os.listdir(file_directory) if (("INSIDe_substances_results" in el) and ("Sampling" not in el))]
sampling_curves_files

['INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW023.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW052.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW022.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW157.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW017.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW143.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW054.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW156.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW048.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW155.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW046.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW211.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_MW064.txt',
 'INSIDe_substances_results_2023_09_18__18_56_37_manhole_RW141.txt',
 'INSIDe_substances_results_2023_0

In [36]:
def tidy_concentration_curve(path):
    # load the complete dataset
    df = pd.read_csv(path, sep="\t")
    df.rename({"time[min]": "time"}, axis=1, inplace=True)

    sampling_location = path.split("_manhole_")[1].replace(".txt", "")

    # identify positions of new subdatasets
    subdataframes = df.loc[df["time"]=="time[min]"]

    # CSB (Chemischer Sauerstoffbedarf) concentration
    df_1 = df.iloc[:subdataframes.index[0]].copy()
    df_1["location"] = sampling_location

    # COV(COV19) concentration 
    df_2 = df.iloc[subdataframes.index[0]:subdataframes.index[1]].copy()
    df_2.columns = df_2.iloc[0]
    df_2 = df_2.iloc[1:]
    df_2.reset_index(drop=True, inplace=True)
    df_2.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_2["location"] = sampling_location

    # T(Temperature) concentration
    df_3 = df.iloc[subdataframes.index[1]:subdataframes.index[2]].copy()
    df_3.columns = df_3.iloc[0]
    df_3 = df_3.iloc[1:]
    df_3.reset_index(drop=True, inplace=True)
    df_3.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_3["location"] = sampling_location

    df_4 = df.iloc[subdataframes.index[2]:].copy()
    df_4.columns = df_4.iloc[0]
    df_4 = df_4.iloc[1:]
    df_4.reset_index(drop=True, inplace=True)
    df_4.rename({"time[min]": "time"}, axis=1, inplace=True)
    df_4["location"] = sampling_location

    # combine the dataframes
    df_1[df_2.columns[1]] = df_2.iloc[:,1]
    df_1[df_3.columns[1]] = df_3.iloc[:,1]
    df_1[df_4.columns[1]] = df_4.iloc[:,1]
    return df_1


In [37]:
df = pd.DataFrame()
for i in range(len(sampling_curves_files)):
    path = f"{file_directory}{sampling_curves_files[i]}"
    df_sub = tidy_concentration_curve(path)
    df = pd.concat((df, df_sub))
df.reset_index(inplace=True, drop=True)

In [38]:
df

,time,CSB(Chemischer Sauerstoffbedarf) concentration [umol/l],location,AFS63(Abfiltrierbare Stoffe) concentration [umol/l],COV(COV19) concentration [umol/l],T(Temperatur) concentration [umol/l]
0,-10,0.0,MW023,0.0,0.0,0.0
1,-5,0.0,MW023,0.0,0.0,0.0
2,0,0.0,MW023,0.0,0.0,0.0
3,5,107.0,MW023,65.3,0.0,0.0
4,10,107.0,MW023,68.3,0.0,0.0
...,...,...,...,...,...,...
15601,4300,107.0,MW061,101.9,0.0,0.0
15602,4305,107.0,MW061,101.9,0.0,0.0
15603,4310,107.0,MW061,101.9,0.0,0.0
15604,4315,107.0,MW061,101.9,0.0,0.0


In [39]:
df.columns = ['time', 'CSB',
       'location', 'AFS63',
       'COV19',
       'Temperature']

"""
['time', 'CSB(Chemischer Sauerstoffbedarf) concentration [umol/l]',
       'location', 'AFS63(Abfiltrierbare Stoffe) concentration [umol/l]',
       'COV(COV19) concentration [umol/l]',
       'T(Temperatur) concentration [umol/l]']
"""

"\n['time', 'CSB(Chemischer Sauerstoffbedarf) concentration [umol/l]',\n       'location', 'AFS63(Abfiltrierbare Stoffe) concentration [umol/l]',\n       'COV(COV19) concentration [umol/l]',\n       'T(Temperatur) concentration [umol/l]']\n"

In [40]:
df["location"].unique()

array(['MW023', 'MW052', 'MW022', 'RW157', 'MW017', 'RW143', 'MW054',
       'RW156', 'MW048', 'RW155', 'MW046', 'RW211', 'MW064', 'RW141',
       'MW059', 'MW043', 'RW126', 'MW061'], dtype=object)

In [41]:
df.to_csv(data_directory+"/concentration_curves.csv", index=False)

### Flow rates

In [47]:
experiment_name

'last_day/MittelstarkerRegen'

In [43]:
file_directory = f"../tandler_output/original/{experiment_name}/"
flow_files = [el for el in os.listdir(file_directory) if "flow_rate" in el]
flow_files = [el for el in flow_files if ("discharge" not in el)]
flow_files[1:3]

['INSIDe_hydraulic_results_2023_09_19__08_31_35_pipe_MW060_flow_rate.txt',
 'INSIDe_hydraulic_results_2023_09_19__08_31_35_pipe_RW217_flow_rate.txt']

In [44]:
def tidy_flow_rate_curve(path):
    # load the complete dataset
    df = pd.read_csv(path, sep="\t")
    df.rename({"t [min] ": "time", " flow rate [l/s]": "flow_rate"}, axis=1, inplace=True)
    df["location"] = path.split("pipe_")[1].split("_flow_rate")[0]
    return df

df = pd.DataFrame()
for i in range(len(flow_files)):
    path = f"{file_directory}{flow_files[i]}"
    df_sub = tidy_flow_rate_curve(path)
    df = pd.concat((df, df_sub))
df.reset_index(inplace=True, drop=True)

In [45]:
df["location"].unique()

array(['MW069', 'MW060', 'RW217', 'MW033', 'MW074', 'RW129', 'MW047',
       'RW128', 'MW070', 'RW136', 'RW130', 'RW135', 'MW059', 'RW132',
       'RW219', 'MW053', 'RW127', 'RW146', 'MW025', 'RW161', 'MW061',
       '456', 'MW054', 'MW056', 'MW055', 'MW073', 'RW153', 'MW071',
       'RW155', 'RW137', 'MW043', 'MW022', 'MW000', 'RW158', 'MW050',
       'MW038', '123', 'MW051', 'MW018', 'MW075', 'RW213', 'RW160',
       'MW072', 'MW063', 'MW014', 'MW048', 'RW221', 'MW057', 'RW133',
       'MW024', 'RW142', 'MW016', 'RW140', 'RW156', 'MW017', 'MW049',
       'RW157', 'RW159', 'MW058', 'RW220', 'MW052', 'MW020', 'MW077',
       'RW143', 'RW139', 'MW045', 'RW215', 'RW138', 'RW145', 'RW154',
       'MW076', 'MW021', 'RW134', '', 'RW224', 'MW046', 'MW029', 'RW218',
       'RW223', 'RW131', 'RW222', 'RW144', 'RW212', 'RW214', 'RW216',
       'RW141', 'RW147', 'RW152', 'MW015', 'MW023'], dtype=object)

In [46]:
df.to_csv(data_directory+"/flow_rates.csv", index=False)